<a href="https://colab.research.google.com/github/klstak/NutriGie/blob/Machine-Learning/Collab/modeling_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
# from google.colab import drive
# drive.mount('/content/drive')

In [47]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dot, Reshape, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [49]:
print("\u2022 Using TensorFlow Version:", tf.__version__)

• Using TensorFlow Version: 2.12.0


In [50]:
# Specify the file path
file_path = '/content/drive/MyDrive/capstone/ML/users.csv'

# Read the CSV file into a DataFrame
data = pd.read_csv(file_path)
data

,user_id,usia,jenis_kelamin,tinggi_badan,berat_badan,aktivitas_fisik,pekerjaan_mahasiswa,pekerjaan_pns,pekerjaan_polisi,pekerjaan_siswa,...,kental,digoreng,direbus,dibakar,dikukus,cepat_saji,beli,ranking,makanan,rating
0,1,21,0,154,55,0,1,0,0,0,...,4,9,7,2,3,1,1,1,63,5
1,1,21,0,154,55,0,1,0,0,0,...,4,9,7,2,3,1,1,2,4,4
2,1,21,0,154,55,0,1,0,0,0,...,4,9,7,2,3,1,1,3,129,3
3,1,21,0,154,55,0,1,0,0,0,...,4,9,7,2,3,1,1,4,145,2
4,1,21,0,154,55,0,1,0,0,0,...,4,9,7,2,3,1,1,5,135,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,60,20,1,177,58,1,0,0,1,0,...,5,8,6,8,8,1,1,1,5,5
1196,60,20,1,177,58,1,0,0,1,0,...,5,8,6,8,8,1,1,2,6,4
1197,60,20,1,177,58,1,0,0,1,0,...,5,8,6,8,8,1,1,3,172,3
1198,60,20,1,177,58,1,0,0,1,0,...,5,8,6,8,8,1,1,4,169,2


# *Food Segment and Average Rating*



In [51]:
# Specify the file path
file_path = '/content/drive/MyDrive/capstone/ML/data_makanan.csv'

# Read the CSV file into a DataFrame
data_makanan = pd.read_csv(file_path)
data_makanan

,no,takaran_g,nama,energi_kkal,protein_g,lemak_g,karbohidrat_g,food_segment
0,1,100 gr,alpukat,85.0,0.900,6.500,7.700,2
1,2,100 gr,apel,58.0,30.120,30.120,14.900,2
2,3,25 gr,bakpia,68.0,2.175,1.675,11.025,2
3,4,1 mangkok,bakso,283.0,17.370,18.430,10.610,3
4,5,50 gr,bakwan,140.0,4.100,5.100,19.500,6
...,...,...,...,...,...,...,...,...
169,170,66 gr,es krim,133.0,2.320,7.080,16.100,6
170,171,1 gelas,jasuke,355.0,9.040,4.450,71.500,8
171,172,257 gr,burger,480.0,22.000,25.000,42.000,1
172,173,60 gr,cimol,133.0,3.130,0.580,24.810,6


In [52]:
data_makanan = data_makanan[["no","food_segment"]].reset_index(drop=True)

In [53]:
mapping_food = dict(data_makanan[['no', 'food_segment']].values)

In [54]:
data["food_segment"] = data["makanan"].map(mapping_food)

In [55]:
#Creating Average of Ratings per User in a Cluster
avg_rating = data.groupby(['user_id','food_segment'])["rating"].mean().to_dict()
data["avg_rating"] = data.set_index(["user_id","food_segment"]).index.map(avg_rating)

In [56]:
data.columns

Index(['user_id', 'usia', 'jenis_kelamin', 'tinggi_badan', 'berat_badan',
       'aktivitas_fisik', 'pekerjaan_mahasiswa', 'pekerjaan_pns',
       'pekerjaan_polisi', 'pekerjaan_siswa', 'pekerjaan_staff',
       'pekerjaan_wirausaha', 'waktu_makan', 'harga_minimal', 'harga_maksimal',
       'asin', 'manis', 'asam', 'gurih', 'pedas', 'renyah', 'halus', 'creamy',
       'kental', 'digoreng', 'direbus', 'dibakar', 'dikukus', 'cepat_saji',
       'beli', 'ranking', 'makanan', 'rating', 'food_segment', 'avg_rating'],
      dtype='object')

#*Separate data based on Meal Time*

In [57]:
# Split the data based on meal time
breakfast_data = data[data['waktu_makan'] == 'sarapan']
lunch_data = data[data['waktu_makan'] == 'makansiang']
dinner_data = data[data['waktu_makan'] == 'makanmalam']
snack_data = data[data['waktu_makan'] == 'snack']

In [58]:
print("breakfast data:", breakfast_data.shape)
print("lunch data:", lunch_data.shape)
print("dinner data:", dinner_data.shape)
print("snack data:", snack_data.shape)

breakfast data: (300, 35)
lunch data: (300, 35)
dinner data: (300, 35)
snack data: (300, 35)


In [61]:
np.unique(data['food_segment']).shape

(8,)

# *Creating User-Item Matrix*

In [63]:
# Create a mapping for user IDs
user_ids = np.unique(data['user_id'])
user_id_map = {user_id: idx for idx, user_id in enumerate(user_ids)}

# Create a mapping for items for each meal time
meal_times = ['breakfast', 'lunch', 'dinner', 'snack']
item_maps = {}

for meal_time, data in zip(meal_times, [breakfast_data, lunch_data, dinner_data, snack_data]):
    items = np.arange(0, 9)  # Adjust the range based on the total number of food items
    item_map = {item: idx for idx, item in enumerate(items)}
    item_maps[meal_time] = item_map

# Calculate the dimensions of the user-item matrices
num_users = len(user_ids)
num_items = max(len(item_map) for item_map in item_maps.values())

# Initialize the user-item matrices for each meal time
user_item_matrices = {meal_time: np.zeros((num_users, num_items)) for meal_time in meal_times}

# Fill in the user-item matrices with ratings for each meal time
for meal_time, data in zip(meal_times, [breakfast_data, lunch_data, dinner_data, snack_data]):
    item_map = item_maps[meal_time]

    for _, row in data.iterrows():
        user_id = row['user_id']
        item = row['food_segment']
        rating = row['avg_rating']
        user_idx = user_id_map[user_id]
        item_idx = item_map[item]
        user_item_matrices[meal_time][user_idx, item_idx] = rating

In [64]:
user_item_matrices

{'breakfast': array([[0.        , 0.        , 0.        , 2.83333333, 0.        ,
         0.        , 0.        , 0.        , 2.875     ],
        [0.        , 3.16666667, 0.        , 4.        , 0.        ,
         0.        , 0.        , 0.        , 2.14285714],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         4.33333333, 0.        , 1.66666667, 3.2       ],
        [0.        , 0.        , 4.        , 0.        , 0.        ,
         0.        , 3.        , 0.        , 2.25      ],
        [0.        , 0.        , 0.        , 2.66666667, 0.        ,
         4.25      , 5.        , 0.        , 2.14285714],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         2.5       , 4.5       , 0.        , 2.33333333],
        [0.        , 2.66666667, 0.        , 1.        , 0.        ,
         0.        , 0.        , 4.        , 2.28571429],
        [0.        , 3.33333333, 0.        , 3.16666667, 0.        ,
         0.        , 0.      

In [65]:
user_item_matrices['lunch']

array([[0.        , 5.        , 0.        , 2.83333333, 0.        ,
        3.        , 0.        , 0.        , 2.875     ],
       [0.        , 3.16666667, 0.        , 0.        , 0.        ,
        0.        , 0.        , 1.        , 2.14285714],
       [0.        , 3.66666667, 0.        , 0.        , 0.        ,
        4.33333333, 1.33333333, 1.66666667, 0.        ],
       [0.        , 3.        , 0.        , 3.33333333, 0.        ,
        0.        , 0.        , 2.6       , 2.25      ],
       [0.        , 0.        , 3.        , 0.        , 0.        ,
        4.25      , 0.        , 0.        , 2.14285714],
       [0.        , 3.5       , 0.        , 0.        , 0.        ,
        0.        , 0.        , 3.25      , 2.33333333],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        3.5       , 0.        , 0.        , 2.28571429],
       [0.        , 3.33333333, 0.        , 3.16666667, 0.        ,
        0.        , 0.        , 0.        , 3.        ],


In [67]:
user_item_matrices['lunch'][0,3]

2.8333333333333335

#  Matrix Factorization

In [68]:
# !pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095456 sha256=b83cffe83d9ef1608e46f37a2da5bdec4d850616bd4b5ae0eecdf2fe2949e855
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [69]:
from surprise import Dataset, Reader, SVD
# Create a Surprise-compatible user-item dataset
user_item_data = []
for meal_time in meal_times:
    user_item_matrix = user_item_matrices[meal_time]
    for user_idx in range(num_users):
        for item_idx in range(num_items):
            rating = user_item_matrix[user_idx, item_idx]
            if rating != 0:
                user_id = list(user_id_map.keys())[list(user_id_map.values()).index(user_idx)]
                item = list(item_map.keys())[list(item_map.values()).index(item_idx)]
                user_item_data.append((user_id, item, rating))

    # Define the Reader object to parse the dataset
    reader = Reader(rating_scale=(1, 5))

    # Load the user-item data into Surprise Dataset
    data = Dataset.load_from_df(pd.DataFrame(user_item_data, columns=['user_id', 'item_id', 'rating']), reader)

    # Create an instance of the SVD algorithm
    svd = SVD()

    # Train the model on the user-item data
    trainset = data.build_full_trainset()
    svd.fit(trainset)

    # Get the estimated ratings for all users and items
    estimated_ratings = np.zeros((num_users, num_items))
    for user_idx in range(num_users):
        for item_idx in range(num_items):
            user_id = list(user_id_map.keys())[list(user_id_map.values()).index(user_idx)]
            item = list(item_map.keys())[list(item_map.values()).index(item_idx)]
            estimated_ratings[user_idx, item_idx] = svd.predict(user_id, item).est
    # Update the user-item matrix for the current meal time with the estimated ratings
    user_item_matrices[meal_time] = np.round(estimated_ratings,decimals=2 )


In [70]:
# Print the completed user-item matrices for each meal time
for meal_time in meal_times:
    print(f"Completed User-Item Matrix for {meal_time}:")
    print(user_item_matrices[meal_time])

Completed User-Item Matrix for breakfast:
[[3.01 3.09 2.88 3.03 3.01 3.15 2.8  3.11 2.92]
 [3.06 3.1  2.86 3.24 3.06 3.08 3.07 3.19 2.86]
 [3.04 3.28 2.98 2.94 3.04 3.44 2.8  3.   2.9 ]
 [3.07 2.94 3.3  3.01 3.07 3.13 2.92 3.17 2.76]
 [3.22 3.25 3.2  3.04 3.22 3.6  3.3  3.21 2.86]
 [3.1  3.11 3.14 3.28 3.1  2.93 3.01 3.09 2.94]
 [2.86 2.95 2.85 2.52 2.86 3.02 2.91 3.21 2.63]
 [3.06 3.29 3.1  3.09 3.06 3.24 2.96 3.25 2.93]
 [3.02 3.08 2.94 2.94 3.02 3.31 3.08 3.25 2.8 ]
 [3.1  3.16 2.78 2.94 3.1  3.22 2.96 3.48 3.26]
 [2.93 3.3  2.81 2.66 2.93 3.14 2.56 3.37 2.89]
 [3.03 3.39 2.85 3.08 3.03 2.69 2.92 3.24 2.97]
 [2.96 3.18 2.89 2.84 2.96 3.05 2.85 3.31 2.92]
 [3.02 2.89 2.99 3.19 3.02 3.25 3.06 3.23 2.83]
 [3.08 3.44 3.08 2.94 3.08 3.29 2.85 3.57 3.06]
 [2.97 3.13 2.96 2.89 2.97 3.11 2.63 3.14 3.07]
 [3.13 3.57 2.99 3.02 3.13 3.38 2.93 3.69 3.04]
 [3.13 3.15 3.26 3.05 3.13 3.25 3.09 3.07 3.06]
 [2.95 3.17 3.12 2.96 2.95 3.13 2.8  3.23 2.77]
 [3.02 3.16 2.98 2.89 3.02 3.14 3.06 3.01 2.88

In [71]:
from sklearn.metrics import mean_squared_error
import math

# Calculate RMSE for each meal time
for meal_time in meal_times:
    user_item_matrix = user_item_matrices[meal_time]

    # Create a list to store the original and estimated ratings
    original_ratings = []
    estimated_ratings = []

    for user_idx in range(num_users):
        for item_idx in range(num_items):
            rating = user_item_matrix[user_idx, item_idx]
            if rating != 0:
                user_id = list(user_id_map.keys())[list(user_id_map.values()).index(user_idx)]
                item = list(item_map.keys())[list(item_map.values()).index(item_idx)]
                original_ratings.append(rating)
                estimated_rating = svd.predict(user_id, item).est
                estimated_ratings.append(estimated_rating)

    # Calculate RMSE
    rmse = math.sqrt(mean_squared_error(original_ratings, estimated_ratings))
    print(f"RMSE for {meal_time}: {rmse}")

RMSE for breakfast: 0.31276819235556896
RMSE for lunch: 0.2658639738946809
RMSE for dinner: 0.21430102675931437
RMSE for snack: 0.00291800067383295


# Modeling per each meal time

In [72]:
#Define the y
y = {}

# Define y as a matrix for each meal time
for meal_time in meal_times:
    user_item_matrix = user_item_matrices[meal_time]
    y[meal_time] = user_item_matrix

In [76]:
#Define the X

X = {}

# Create X for each meal time
X['breakfast'] = breakfast_data.drop(columns=["user_id","makanan","rating","ranking",'waktu_makan','food_segment', 'avg_rating']).drop_duplicates()
X['lunch'] = lunch_data.drop(columns=["user_id","makanan","rating","ranking",'waktu_makan','food_segment', 'avg_rating']).drop_duplicates()
X['dinner'] = dinner_data.drop(columns=["user_id","makanan","rating","ranking",'waktu_makan','food_segment', 'avg_rating']).drop_duplicates()
X['snack'] = snack_data.drop(columns=["user_id","makanan","rating","ranking",'waktu_makan','food_segment', 'avg_rating']).drop_duplicates()

In [77]:
X['snack'].columns

Index(['usia', 'jenis_kelamin', 'tinggi_badan', 'berat_badan',
       'aktivitas_fisik', 'pekerjaan_mahasiswa', 'pekerjaan_pns',
       'pekerjaan_polisi', 'pekerjaan_siswa', 'pekerjaan_staff',
       'pekerjaan_wirausaha', 'harga_minimal', 'harga_maksimal', 'asin',
       'manis', 'asam', 'gurih', 'pedas', 'renyah', 'halus', 'creamy',
       'kental', 'digoreng', 'direbus', 'dibakar', 'dikukus', 'cepat_saji',
       'beli'],
      dtype='object')

In [78]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# Normalize each X for each meal time
for meal_time, X_data in X.items():
    X_norm = scaler.fit_transform(X_data)
    X[meal_time][X_data.columns] = X_norm


Scenario 1:  Training and evaluating separate models for each meal time, and it iterates over the meal times to perform the same steps for each model

In [80]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
# import scipy
# import numpy as np
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Dense
# from tensorflow.keras.optimizers import Adam

# meal_times = ['breakfast', 'lunch', 'dinner', 'snack']
# precision = {meal_time: [] for meal_time in meal_times}
# recall = {meal_time: [] for meal_time in meal_times}
# f1 = {meal_time: [] for meal_time in meal_times}
# mae_list = {meal_time: [] for meal_time in meal_times}

# for meal_time in meal_times:
#     X_meal = X[meal_time]
#     y_meal = y[meal_time]

#     X_train, X_test, y_train, y_test = train_test_split(X_meal, y_meal, test_size=0.2, random_state=40)
#     print('Train size: {train}, Test size: {test}'.format(train=X_train.shape[0], test=X_test.shape[0]))

#     INPUT_LENGTH = X_train.shape[1]  # 784
#     HIDDEN_UNITS = 1000

#     # Define the input layer
#     inputs = Input(shape=(INPUT_LENGTH,))

#     # Define the hidden layer
#     hidden_layer = Dense(HIDDEN_UNITS, activation='relu')(inputs)

#     # Define the output layer
#     outputs = Dense(y_train.shape[1], activation='linear')(hidden_layer)

#     # Create the model
#     model = Model(inputs=inputs, outputs=outputs)

#     # Compile the model
#     model.compile(optimizer=Adam(), loss='mean_squared_error')

#     # Train the model
#     model.fit(X_train, y_train, epochs=30, batch_size=32)

#     # Predict on the test set
#     predicted_ratings = model.predict(X_test)

#     for i in range(0, y_test.shape[0]):
#         user_preferences = np.array(X_test)[i, :]
#         y_label = y_test[i, :]

#         # Get top N recommended items based on predicted ratings
#         top_n = 5
#         recommended_items = np.argsort(predicted_ratings[i])[-top_n:][::-1]

#         # Get top N actual items
#         actual_items = np.argsort(y_label)[-top_n:][::-1]

#         tp = []
#         relevant = []
#         for i in range(0, top_n):
#             if np.isin(recommended_items[i], actual_items) == True:
#                 tp.append(1)
#                 relevant.append(1)
#             else:
#                 tp.append(0)
#         p = np.mean(tp)
#         precision[meal_time].append(p)
#         try:
#             r = sum(tp) / len(relevant)
#         except:
#             r = 1
#         recall[meal_time].append(r)
#         f1[meal_time].append(2 * p * r / (p + r))

#         # Calculate MAE
#         mae = np.mean(np.abs(predicted_ratings[i] - y_label))
#         mae_list[meal_time].append(mae)

#     avg_precision = np.mean(precision[meal_time])
#     avg_recall = np.mean(recall[meal_time])
#     avg_f1 = np.mean(f1[meal_time])
#     avg_mae = np.mean(mae_list[meal_time])

#     print("Meal Time: " + meal_time)
#     print("Average Precision@5 = " + str(np.round(avg_precision * 100, 2)) + "%")
#     print("Average Recall@5 = " + str(np.round(avg_recall * 100, 2)) + "%")
#     print("Average F1@5 = " + str(np.round(avg_f1 * 100, 2)) + "%")
#     print("Mean Absolute Error (MAE) = " + str(avg_mae))
#     print()

Train size: 48, Test size: 12
Epoch 1/30
2/2 [==============================] - 1s 13ms/step - loss: 9.0255
Epoch 2/30
2/2 [==============================] - 0s 7ms/step - loss: 7.5235
Epoch 3/30
2/2 [==============================] - 0s 8ms/step - loss: 6.0854
Epoch 4/30
2/2 [==============================] - 0s 9ms/step - loss: 4.6948
Epoch 5/30
2/2 [==============================] - 0s 14ms/step - loss: 3.3691
Epoch 6/30
2/2 [==============================] - 0s 8ms/step - loss: 2.1569
Epoch 7/30
2/2 [==============================] - 0s 7ms/step - loss: 1.1838
Epoch 8/30
2/2 [==============================] - 0s 8ms/step - loss: 0.5802
Epoch 9/30
2/2 [==============================] - 0s 8ms/step - loss: 0.4475
Epoch 10/30
2/2 [==============================] - 0s 8ms/step - loss: 0.6649
Epoch 11/30
2/2 [==============================] - 0s 8ms/step - loss: 0.8966
Epoch 12/30
2/2 [==============================] - 0s 8ms/step - loss: 0.9258
Epoch 13/30
2/2 [========================

In [ ]:
# Train size: 48, Test size: 12
# Epoch 1/30
# 2/2 [==============================] - 1s 13ms/step - loss: 9.0255
# Epoch 2/30
# 2/2 [==============================] - 0s 7ms/step - loss: 7.5235
# Epoch 3/30
# 2/2 [==============================] - 0s 8ms/step - loss: 6.0854
# Epoch 4/30
# 2/2 [==============================] - 0s 9ms/step - loss: 4.6948
# Epoch 5/30
# 2/2 [==============================] - 0s 14ms/step - loss: 3.3691
# Epoch 6/30
# 2/2 [==============================] - 0s 8ms/step - loss: 2.1569
# Epoch 7/30
# 2/2 [==============================] - 0s 7ms/step - loss: 1.1838
# Epoch 8/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.5802
# Epoch 9/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.4475
# Epoch 10/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.6649
# Epoch 11/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.8966
# Epoch 12/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.9258
# Epoch 13/30
# 2/2 [==============================] - 0s 9ms/step - loss: 0.7561
# Epoch 14/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.5349
# Epoch 15/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.3749
# Epoch 16/30
# 2/2 [==============================] - 0s 12ms/step - loss: 0.3001
# Epoch 17/30
# 2/2 [==============================] - 0s 10ms/step - loss: 0.3114
# Epoch 18/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.3368
# Epoch 19/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.3591
# Epoch 20/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.3526
# Epoch 21/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.3195
# Epoch 22/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.2781
# Epoch 23/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.2377
# Epoch 24/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.2157
# Epoch 25/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.2123
# Epoch 26/30
# 2/2 [==============================] - 0s 10ms/step - loss: 0.2157
# Epoch 27/30
# 2/2 [==============================] - 0s 14ms/step - loss: 0.2189
# Epoch 28/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.2116
# Epoch 29/30
# 2/2 [==============================] - 0s 9ms/step - loss: 0.1985
# Epoch 30/30
# 2/2 [==============================] - 0s 7ms/step - loss: 0.1816
# 1/1 [==============================] - 0s 85ms/step
# Meal Time: breakfast
# Average Precision@5 = 75.0%
# Average Recall@5 = 100.0%
# Average F1@5 = 84.85%
# Mean Absolute Error (MAE) = 0.19774224404935484

# Train size: 48, Test size: 12
# Epoch 1/30
# 2/2 [==============================] - 1s 9ms/step - loss: 9.1753
# Epoch 2/30
# 2/2 [==============================] - 0s 7ms/step - loss: 7.5613
# Epoch 3/30
# 2/2 [==============================] - 0s 10ms/step - loss: 6.0224
# Epoch 4/30
# 2/2 [==============================] - 0s 9ms/step - loss: 4.5469
# Epoch 5/30
# 2/2 [==============================] - 0s 11ms/step - loss: 3.1612
# Epoch 6/30
# 2/2 [==============================] - 0s 7ms/step - loss: 1.9276
# Epoch 7/30
# 2/2 [==============================] - 0s 9ms/step - loss: 0.9920
# Epoch 8/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.4744
# Epoch 9/30
# 2/2 [==============================] - 0s 9ms/step - loss: 0.4590
# Epoch 10/30
# 2/2 [==============================] - 0s 7ms/step - loss: 0.8254
# Epoch 11/30
# 2/2 [==============================] - 0s 9ms/step - loss: 1.0369
# Epoch 12/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.9885
# Epoch 13/30
# 2/2 [==============================] - 0s 9ms/step - loss: 0.7534
# Epoch 14/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.5301
# Epoch 15/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.3640
# Epoch 16/30
# 2/2 [==============================] - 0s 9ms/step - loss: 0.3307
# Epoch 17/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.3508
# Epoch 18/30
# 2/2 [==============================] - 0s 7ms/step - loss: 0.3910
# Epoch 19/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.4070
# Epoch 20/30
# 2/2 [==============================] - 0s 7ms/step - loss: 0.3933
# Epoch 21/30
# 2/2 [==============================] - 0s 7ms/step - loss: 0.3547
# Epoch 22/30
# 2/2 [==============================] - 0s 14ms/step - loss: 0.3058
# Epoch 23/30
# 2/2 [==============================] - 0s 12ms/step - loss: 0.2700
# Epoch 24/30
# 2/2 [==============================] - 0s 20ms/step - loss: 0.2609
# Epoch 25/30
# 2/2 [==============================] - 0s 16ms/step - loss: 0.2647
# Epoch 26/30
# 2/2 [==============================] - 0s 15ms/step - loss: 0.2633
# Epoch 27/30
# 2/2 [==============================] - 0s 21ms/step - loss: 0.2567
# Epoch 28/30
# 2/2 [==============================] - 0s 18ms/step - loss: 0.2435
# Epoch 29/30
# 2/2 [==============================] - 0s 14ms/step - loss: 0.2268
# Epoch 30/30
# 2/2 [==============================] - 0s 17ms/step - loss: 0.2178
# 1/1 [==============================] - 0s 55ms/step
# Meal Time: lunch
# Average Precision@5 = 63.33%
# Average Recall@5 = 100.0%
# Average F1@5 = 76.98%
# Mean Absolute Error (MAE) = 0.19408455230571606

# Train size: 48, Test size: 12
# Epoch 1/30
# 2/2 [==============================] - 1s 8ms/step - loss: 8.8129
# Epoch 2/30
# 2/2 [==============================] - 0s 10ms/step - loss: 7.3138
# Epoch 3/30
# 2/2 [==============================] - 0s 10ms/step - loss: 5.8728
# Epoch 4/30
# 2/2 [==============================] - 0s 8ms/step - loss: 4.5199
# Epoch 5/30
# 2/2 [==============================] - 0s 7ms/step - loss: 3.2041
# Epoch 6/30
# 2/2 [==============================] - 0s 10ms/step - loss: 2.0159
# Epoch 7/30
# 2/2 [==============================] - 0s 10ms/step - loss: 1.0867
# Epoch 8/30
# 2/2 [==============================] - 0s 10ms/step - loss: 0.5281
# Epoch 9/30
# 2/2 [==============================] - 0s 11ms/step - loss: 0.4486
# Epoch 10/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.6550
# Epoch 11/30
# 2/2 [==============================] - 0s 9ms/step - loss: 0.8520
# Epoch 12/30
# 2/2 [==============================] - 0s 11ms/step - loss: 0.8649
# Epoch 13/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.6989
# Epoch 14/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.5178
# Epoch 15/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.3696
# Epoch 16/30
# 2/2 [==============================] - 0s 15ms/step - loss: 0.3149
# Epoch 17/30
# 2/2 [==============================] - 0s 10ms/step - loss: 0.3326
# Epoch 18/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.3680
# Epoch 19/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.3835
# Epoch 20/30
# 2/2 [==============================] - 0s 11ms/step - loss: 0.3769
# Epoch 21/30
# 2/2 [==============================] - 0s 10ms/step - loss: 0.3462
# Epoch 22/30
# 2/2 [==============================] - 0s 10ms/step - loss: 0.3060
# Epoch 23/30
# 2/2 [==============================] - 0s 7ms/step - loss: 0.2841
# Epoch 24/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.2601
# Epoch 25/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.2590
# Epoch 26/30
# 2/2 [==============================] - 0s 9ms/step - loss: 0.2636
# Epoch 27/30
# 2/2 [==============================] - 0s 10ms/step - loss: 0.2664
# Epoch 28/30
# 2/2 [==============================] - 0s 13ms/step - loss: 0.2568
# Epoch 29/30
# 2/2 [==============================] - 0s 7ms/step - loss: 0.2430
# Epoch 30/30
# 2/2 [==============================] - 0s 7ms/step - loss: 0.2299
# 1/1 [==============================] - 0s 94ms/step
# Meal Time: dinner
# Average Precision@5 = 71.67%
# Average Recall@5 = 100.0%
# Average F1@5 = 82.87%
# Mean Absolute Error (MAE) = 0.26567220935115105

# Train size: 48, Test size: 12
# Epoch 1/30
# 2/2 [==============================] - 1s 8ms/step - loss: 8.9454
# Epoch 2/30
# 2/2 [==============================] - 0s 9ms/step - loss: 7.4003
# Epoch 3/30
# 2/2 [==============================] - 0s 7ms/step - loss: 5.9895
# Epoch 4/30
# 2/2 [==============================] - 0s 8ms/step - loss: 4.6391
# Epoch 5/30
# 2/2 [==============================] - 0s 11ms/step - loss: 3.3475
# Epoch 6/30
# 2/2 [==============================] - 0s 7ms/step - loss: 2.1783
# Epoch 7/30
# 2/2 [==============================] - 0s 10ms/step - loss: 1.2360
# Epoch 8/30
# 2/2 [==============================] - 0s 13ms/step - loss: 0.6575
# Epoch 9/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.5003
# Epoch 10/30
# 2/2 [==============================] - 0s 7ms/step - loss: 0.7027
# Epoch 11/30
# 2/2 [==============================] - 0s 10ms/step - loss: 0.9471
# Epoch 12/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.9748
# Epoch 13/30
# 2/2 [==============================] - 0s 11ms/step - loss: 0.8078
# Epoch 14/30
# 2/2 [==============================] - 0s 11ms/step - loss: 0.6077
# Epoch 15/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.4376
# Epoch 16/30
# 2/2 [==============================] - 0s 9ms/step - loss: 0.3683
# Epoch 17/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.3599
# Epoch 18/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.3803
# Epoch 19/30
# 2/2 [==============================] - 0s 9ms/step - loss: 0.3904
# Epoch 20/30
# 2/2 [==============================] - 0s 10ms/step - loss: 0.3768
# Epoch 21/30
# 2/2 [==============================] - 0s 9ms/step - loss: 0.3403
# Epoch 22/30
# 2/2 [==============================] - 0s 7ms/step - loss: 0.2998
# Epoch 23/30
# 2/2 [==============================] - 0s 7ms/step - loss: 0.2677
# Epoch 24/30
# 2/2 [==============================] - 0s 10ms/step - loss: 0.2516
# Epoch 25/30
# 2/2 [==============================] - 0s 9ms/step - loss: 0.2649
# Epoch 26/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.2668
# Epoch 27/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.2593
# Epoch 28/30
# 2/2 [==============================] - 0s 8ms/step - loss: 0.2432
# Epoch 29/30
# 2/2 [==============================] - 0s 7ms/step - loss: 0.2285
# Epoch 30/30
# 2/2 [==============================] - 0s 7ms/step - loss: 0.2179
# 1/1 [==============================] - 0s 75ms/step
# Meal Time: snack
# Average Precision@5 = 78.33%
# Average Recall@5 = 100.0%
# Average F1@5 = 87.04%
# Mean Absolute Error (MAE) = 0.24270849810706244

